### 1 - Import test data

In [5]:
import pandas as pd
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print(algo_dir)
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [7]:
df=dataset[dataset.High!=dataset.Low]
len(df)

73034

In [8]:
df["EMA_slow"]=ta.ema(df.Close, length=50)
df["EMA_fast"]=ta.ema(df.Close, length=30)
df['RSI']=ta.rsi(df.Close, length=10)
my_bbands = ta.bbands(df.Close, length=15, std=1.5)
df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
df=df.join(my_bbands)
df

,Open,High,Low,Close,Volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
Date,,,,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02 14:05:00,1269.2,1273.4,1269.2,1270.8,22701,1268.125325,1264.495469,75.023492,3.137828,1257.900763,1262.646667,1267.392570,0.751739,1.358986
2024-08-02 14:10:00,1270.9,1271.6,1268.9,1271.2,12412,1268.245901,1264.928019,75.699880,3.075281,1257.472959,1263.200000,1268.927041,0.906751,1.198441
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,14456,1268.499003,1265.558470,80.764377,3.507384,1257.094831,1264.140000,1271.185169,1.114618,1.249450


In [9]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0


df=df[-10000:-1]
from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

ModuleNotFoundError: No module named 'tqdm'

In [7]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 9998/9998 [00:27<00:00, 364.09it/s]


In [9]:
df[df.TotalSignal != 0].head(20)

,index,Gmt time,Open,High,Low,Close,Volume,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal
3,3,2022-08-15 03:25:00,1.02478,1.02492,1.02474,1.02487,723.19,1.025308,1.025030,51.979645,0.000268,1.024246,1.024556,1.024866,0.060462,1.006890,1,1
7,7,2022-08-15 03:45:00,1.02491,1.02523,1.02489,1.02508,569.41,1.025250,1.025005,59.462538,0.000257,1.024224,1.024621,1.025019,0.077612,1.076775,1,1
8,8,2022-08-15 03:50:00,1.02508,1.02525,1.02504,1.02525,370.81,1.025250,1.025021,64.530972,0.000251,1.024225,1.024677,1.025129,0.088246,1.134054,1,1
9,9,2022-08-15 03:55:00,1.02523,1.02533,1.02518,1.02523,591.27,1.025249,1.025034,63.493245,0.000236,1.024276,1.024741,1.025206,0.090773,1.026057,1,1
11,11,2022-08-15 04:05:00,1.02519,1.02534,1.02507,1.02534,740.19,1.025250,1.025063,65.954393,0.000244,1.024458,1.024878,1.025298,0.081879,1.050553,1,1
13,13,2022-08-15 04:15:00,1.02530,1.02552,1.02525,1.02549,366.86,1.025261,1.025104,69.084832,0.000237,1.024590,1.025001,1.025412,0.080210,1.094371,1,1
14,14,2022-08-15 04:20:00,1.02549,1.02559,1.02544,1.02549,217.41,1.025270,1.025129,69.084832,0.000225,1.024660,1.025065,1.025469,0.078979,1.025374,1,1
16,16,2022-08-15 04:30:00,1.02533,1.02558,1.02531,1.02555,644.86,1.025283,1.025168,66.376842,0.000228,1.024781,1.025163,1.025545,0.074544,1.005976,1,1
53,53,2022-08-15 07:35:00,1.02405,1.02446,1.02395,1.02415,1615.68,1.024400,1.024132,53.887351,0.000575,1.023421,1.023782,1.024143,0.070613,1.009041,1,1
112,112,2022-08-15 12:30:00,1.02026,1.02149,1.02026,1.02095,3551.63,1.020541,1.020083,68.363481,0.000664,1.019117,1.019821,1.020526,0.138127,1.301238,1,1


In [7]:
import numpy as np
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

C:\Users\Chinook\AppData\Local\Temp\ipykernel_13224\2345118794.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)


In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [9]:
def SIGNAL():
    return df.TotalSignal

In [10]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 3000
    slcoef = 1.1
    TPSLRatio = 1.5
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=250, margin=1/30)

f:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

C:\Users\Chinook\AppData\Local\Temp\ipykernel_13224\2590162779.py:36: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



In [12]:
bt.run()

Start                                     0.0
End                                   29998.0
Duration                              29998.0
Exposure Time [%]                    32.59442
Equity Final [$]                   564.111374
Equity Peak [$]                    568.743962
Return [%]                          125.64455
Buy & Hold Return [%]               -6.960914
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -16.861386
Avg. Drawdown [%]                   -1.204402
Max. Drawdown Duration                 3827.0
Avg. Drawdown Duration              143.12234
# Trades                               1671.0
Win Rate [%]                        43.985637
Best Trade [%]                        0.40522
Worst Trade [%]                     -0.252185
Avg. Trade [%]                    

In [14]:
bt.plot()

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_drag' property; using the latest value

f:\Python\Lib\site-packages\backtesting\_plotting.py:659: UserWarning:

found multiple competing values for 'toolbar.active_scroll' property; using the latest value



GridPlot(id='p1966', ...)